In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import sys
sys.path.append('/home/simon/Research/lib/caffe/python/')
import caffe
caffe.set_device(0)
caffe.set_mode_gpu()
import Caffe
import Resize
import Pad
import PIL
import Classification
import glob
import pyprind
import os

import matplotlib
%matplotlib inline  
import matplotlib.pyplot as plt
import Dataset
import Evaluation
import ImageReader
import Crop
import Classificator
import sklearn
import scipy
import Lambda
import Blob

In [ ]:
if True:
    # Read dataset from image, label, and split list
    basedir = '/home/simon/Datasets/CUB_200_2011/'
    d = Dataset.Dataset()
    d.read_from_file('%s/imagelist_absolute.txt'%basedir, 'imagepaths','string')
    d.read_from_file('%s/labels.txt'%basedir,'labels','int')
    d.read_from_file('%s/tr_ID.txt'%basedir,'split_assignments','int')
else:
    # Read dataset from directory structure
    d = Dataset.Dataset()
    d.use_images_in_folder('/home/simon/Datasets/dtd/images/')
    d.create_labels_from_path()
    d.make_random_split()

In [ ]:
c = Classification.Classification()
c.add_algorithm(ImageReader.ImageReader())
s = 224
#c.add_algorithm(Resize.Resize((s,s), mode='stretch'))
c.add_algorithm(Resize.Resize((s,), mode='resize_smaller_side'))
c.add_algorithm(Crop.Crop((s,s),'center'))
#c.add_algorithm(FlipAugmentation())
#c.add_algorithm(Pad.Pad(s,s))
modeldir = '/home/simon/Data/caffe/alexnet/'
c.add_algorithm(Caffe.Caffe(modeldir + '/deploy.prototxt',
                           modeldir + '/model',
                           #mean = np.float32([0,0,0]),
                           outblob = 'fc7',
                           batchsize = 5))

c.add_algorithm(Lambda.Lambda(lambda x: sklearn.preprocessing.normalize(x.reshape((1,-1)))))
c.add_algorithm(Classificator.Classificator(sklearn.svm.LinearSVC(C=1),False))

In [ ]:
if True:
    ev = Evaluation.Evaluation()
    ev.fixed_split_eval(d, c)
elif True:
    ev = Evaluation.Evaluation()
    ev.random_split_eval(d,c,runs=1,absolute_train_per_class=1)

### Compute features

In [ ]:
train_ids = np.where(d.split_assignments)[0]
test_ids = np.where(np.invert(np.array(d.split_assignments).astype(bool)))[0]

In [ ]:
#Calc the features
def calc_features(pipeline, images):
    out_shape = (len(images), ) + pipeline.compute(images[0]).shape
    feats = np.zeros(tuple(out_shape),dtype=np.float32)
    feats[:] = 1
    print('feats will occupy %.4f GiB'%(feats.nbytes/1024/1024/1024))
    for idx,impath in enumerate(pyprind.prog_bar(images, update_interval=2)):
        feats[idx,...] = c.compute(impath)
    return feats

In [ ]:
train_fv = calc_features(c, np.array(d.imagepaths)[train_ids])
test_fv = calc_features(c, np.array(d.imagepaths)[test_ids])

In [ ]:
test_fv = calc_features(c, np.array(d.imagepaths)[test_ids][:100])

In [ ]:
task = dict(train_feats=train_fv.reshape((len(train_fv),-1)),
                                test_feats=test_fv.reshape((len(test_fv),-1)),
                                train_Y=np.array(d.labels)[train_ids],
                                test_Y=np.array(d.labels)[test_ids])

if False:
    scaler = sklearn.preprocessing.RobustScaler().fit(task['train_feats'])
    task['train_feats'] = scaler.transform(task['train_feats'])
    task['test_feats'] = scaler.transform(task['test_feats'])
elif False:
    task['train_feats'] = sklearn.preprocessing.normalize(task['train_feats'])
    task['test_feats'] = sklearn.preprocessing.normalize(task['test_feats'])

In [ ]:
# for comparison: explicit feature transform of linear kernel
model = sklearn.svm.LinearSVC(C=1)
model.fit(task['train_feats'],task['train_Y'])
model.score(task['test_feats'],task['test_Y'])

### Cross validation

In [ ]:
import sklearn.model_selection
import sklearn.metrics 

In [ ]:
#Calc the features
def calc_features(pipeline, images):
    out_shape = (len(images), ) + pipeline.compute(images[0]).shape
    feats = np.zeros(tuple(out_shape),dtype=np.float32)
    feats[:] = 1
    print('feats will occupy %.4f GiB'%(feats.nbytes/1024/1024/1024))
    for idx,impath in enumerate(pyprind.prog_bar(images, update_interval=2)):
        feats[idx,...] = c.compute(impath)
    return feats

In [ ]:
all_feats = calc_features(c, d.imagepaths)
all_feats = all_feats.reshape((len(d.labels),-1))

In [ ]:
model = sklearn.svm.LinearSVC(C=1)

In [ ]:
def compute_score(estimator, X, y):
        cm = sklearn.metrics.confusion_matrix(y, estimator.predict(X))
        acc = cm.diagonal().sum()/cm.ravel().sum()
        cm = cm / cm.sum(axis=1,keepdims=True)
        mAP = np.nanmean(cm.diagonal())
        return mAP

In [ ]:
scores = []
for i in pyprind.prog_bar(range(100)):
    d.split_assignments = []
    d.fill_split_assignments(0)
    d.make_random_split(absolute_per_class=1, source_value=0, target_value=1)
    train_ids = np.where(d.split_assignments)[0]
    test_ids = np.where(np.invert(np.array(d.split_assignments).astype(bool)))[0]
    model.fit(all_feats[train_ids], np.array(d.labels)[train_ids])
    scores.append(compute_score(model, all_feats[test_ids], np.array(d.labels)[test_ids]))

In [ ]:
np.mean(scores)

### Other stuff

In [ ]:
test_images = [im for im,split in zip(d.imagepaths,d.split_assignments) if split==0]
test_labels = [label for label,split in zip(d.labels,d.split_assignments) if split==0]

In [ ]:
predictions = c.compute_all(test_images)